In [2]:
import sys
print(sys.path)
sys.path.append('/Users/leehojin/Projects/new_automation/')

['/opt/homebrew/Caskroom/miniconda/base/lib/python313.zip', '/opt/homebrew/Caskroom/miniconda/base/lib/python3.13', '/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/lib-dynload', '', '/Users/leehojin/Projects/new_automation/.venv/lib/python3.13/site-packages', '/Users/leehojin/Projects/new_automation/']


In [7]:
import os
import logging
from dotenv import load_dotenv
load_dotenv()

from playwright.async_api import async_playwright
from src.crawler.login import Login
from src.crawler.navigator import Navigator
from src.crawler.popup_handler import PopupHandler

#로깅
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# 환경변수 확인
url = os.getenv("WEB_SITE_URL")
username = os.getenv("ID")
password = os.getenv("PW")
print(url, username, password)

pw = await async_playwright().start()
browser = await pw.chromium.launch(
    headless=False,  # 화면 켜기
    slow_mo=500,
)
page = await browser.new_page()

print("브라우저 준비 완료")

auth = Login(
    page,
    url,
    username,
    password
)
await auth.login()

http://admin.gobermuda.net:8080/ gobermuda@gmail.com tjfgus!@
브라우저 준비 완료


2025-12-17 18:47:17,557 - src.crawler.login - INFO - 로그인 시작
2025-12-17 18:47:19,929 - src.crawler.login - INFO - 로그인 성공


In [10]:
# popup 순서를 닫는 버튼으로 반환
handler = PopupHandler(page)
popup_list = await handler.get_sorted_close_buttons()
print(f"✓ 팝업 종류: {popup_list}")

#팝업 닫기
closed = await handler.close_all_popups()
print(f"{closed}개의 팝업을 닫았습니다.")

✓ 팝업 종류: ['button.swal2-confirm', 'button.swal2-cancel', '.guide-modal-close']


2025-12-17 18:50:53,688 - src.crawler.popup_handler - INFO - 팝업 닫기 성공: button.swal2-confirm
2025-12-17 18:50:54,769 - src.crawler.popup_handler - INFO - 팝업 닫기 성공: .guide-modal-close
2025-12-17 18:50:55,318 - src.crawler.popup_handler - INFO - 총 2개의 팝업을 닫았습니다.


2개의 팝업을 닫았습니다.


In [4]:
# 사이드바 열기
await navigation.open_sidebar()
await page.wait_for_timeout(300)
await navigation.click_police_button()

In [7]:
# 데이터가 로드될 때까지 대기
await page.wait_for_selector('.police-table-row', state='visible')

table_data = await page.evaluate("""
() => {
    const rows = Array.from(document.querySelectorAll('.police-table-row'));
    return rows.map(row => ({
        id: row.querySelector('.police-table-no')?.textContent?.trim() || '',
        type: row.querySelector('.police-table-type')?.textContent?.trim() || '',
        fbUid: row.querySelector('.police-table-uid')?.textContent?.trim() || '',
        nick: row.querySelector('.police-table-nick')?.textContent?.trim() || '',
        country: row.querySelector('.police-table-country')?.textContent?.trim() || '',
        gender: row.querySelector('.police-table-gender')?.textContent?.trim() || '',
        lastLogin: row.querySelector('.police-table-login')?.textContent?.trim() || '',
        captureLink: row.querySelector('.police-table-clink a')?.href || null
    }));
}
""")

print(f"총 {len(table_data)}개의 데이터 행을 찾았습니다.")

총 1335개의 데이터 행을 찾았습니다.


In [14]:
from datetime import datetime, timedelta, timezone
import re

# 한국 시간 기준 (UTC+9)
now = datetime.now(timezone.utc).astimezone(timezone(timedelta(hours=9)))

# 이번 주 월요일 (00:00:00)
this_week_monday = now - timedelta(days=now.weekday())
this_week_monday = this_week_monday.replace(hour=0, minute=0, second=0, microsecond=0)

# 지난 주 월요일 (00:00:00)
start_of_week = this_week_monday - timedelta(days=7)

print("[필터 기준]")
print(f"지난주 월요일 이후: {start_of_week}")

# 날짜 파싱 처리 함수
def parse_last_login(raw):
    raw = raw.replace("오전", "AM").replace("오후", "PM")
    raw = re.sub(r"\s+", " ", raw)
    
    try:
        return datetime.strptime(raw, "%Y. %m. %d. %p %I:%M:%S").replace(tzinfo=timezone(timedelta(hours=9)))
    except Exception:
        return None


ph_count = 0
invalid_date_count = 0
out_of_range_count = 0

filtered_data = []

for row in table_data:
    if row["id"].lower() == "id":
        continue

    last_login = parse_last_login(row["lastLogin"])
    if not last_login:
        invalid_date_count += 1
        continue

    # PH 제외
    if row["country"] == "PH":
        ph_count += 1
        continue

    # 지난주 월요일 이후 데이터만 포함
    if last_login < start_of_week:
        out_of_range_count += 1
        continue

    filtered_data.append(row)

print("필터링 결과:")
print(f"- 총 데이터: {len(table_data)}개")
print(f"- 필터링 후: {len(filtered_data)}개")
print("- 제외된 데이터:")
print(f"  • PH 국가: {ph_count}개")
print(f"  • 지난주 이전 날짜: {out_of_range_count}개")
print(f"  • 날짜 파싱 실패: {invalid_date_count}개")


[필터 기준]
지난주 월요일 이후: 2025-11-24 00:00:00+09:00
필터링 결과:
- 총 데이터: 1335개
- 필터링 후: 473개
- 제외된 데이터:
  • PH 국가: 41개
  • 지난주 이전 날짜: 820개
  • 날짜 파싱 실패: 0개
